### Assignment - IO

In [17]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import linearmodels as ln
from statsmodels.sandbox.regression.gmm import IV2SLS
from scipy.optimize import minimize
from scipy import ndimage
from pystout import pystout
from tabulate import tabulate
from scipy.stats import uniform

In [18]:
df = pd.read_csv("autodata.csv")
df.rename(columns = {'Firm ID (there is a different number for each firm)':'Firm'}, inplace = True)
df.rename(columns = {'Price($1000) (list price)':'Price'}, inplace = True)

In [19]:
pd.unique(df['Firm'])

array([ 3, 20,  7,  8, 19, 16, 18, 21,  2, 13, 22,  1,  4,  9, 10, 11,  5,
        6, 23, 12], dtype=int64)

In [20]:
#Calculate shares
s0 = 0.6 #Set outside good share
df['Total q'] = df['Quantity'].sum()
df['Share'] = df['Quantity'] / df['Total q'] * (1 - s0)

In [21]:
#Add outside good:
outside = {'Name':'outside', 'Air Conditioning':0, 'Horsepower':0, 'Weight of Car':0, 'Price':0, 'Quantity':0, 'Share':s0}
df = df.append(outside, ignore_index=True)

In [22]:
#And sort data
df.sort_values(by=['Price'], ascending = False, inplace=True) #Sort dataset
df.reset_index(inplace = True)

In [23]:
#Calculate thresholds
sN = df['Share'].iloc[0]
#General formatof inverse of Uniform cdf: F(p) = a + p(b - a)
#If we assume a = 0 and b = 1
#We will always maintain a = 0 (idea: lowest possible taste for quality is not caring)
b = 10
df['Δ'] = uniform.ppf(1 - sN)

for i, r in df.iloc[1:].iterrows():
    Δ_previous = df['Δ'].iloc[i-1]
    s = df['Share'].iloc[i]
    
    df['Δ'].iloc[i] = uniform.ppf(uniform.cdf(Δ_previous) - s)

In [24]:
#Resort in opposite way to get quality
df.sort_values(by=['Price'], inplace=True) #Sort dataset
df.reset_index(inplace = True)

In [25]:
#Calculate implied quality levels
df['δ'] = 0

for i, r in df.iloc[1:].iterrows():
    δ_previous = df['δ'].iloc[i-1]
    p_previous = df['Price'].iloc[i-1]
    p = df['Price'].iloc[i]
    Δ = df['Δ'].iloc[i]
    
    df['δ'].iloc[i] = δ_previous + (p - p_previous) / Δ

In [26]:
# Run OLS regression
x = df[['Air Conditioning', 'Weight of Car', 'Horsepower']]
y = df['δ']
model1 = sm.OLS(y, x).fit()
model1_α = model1.params

In [27]:
pystout(models=[model1],
        file='OLS_demand.tex',
        digits=3,
        endog_names=False,
        exogvars=['Air Conditioning', 'Weight of Car', 'Horsepower'],
        stars=False,
        modstat={'nobs':'Obs','rsquared_adj':'Adj. R\sym{2}'}
        )

In [28]:
#Estimate parameters under marginal cost pricing
x = df[['Air Conditioning', 'Weight of Car', 'Horsepower', 'Quantity']]
y = df['Price']
supply1 = sm.OLS(y, x).fit()

In [13]:
#Estimate parameters under firms setting prices in a NE
df['partials'] = 0
N = df.shape[0]

for i, r in df.iloc[1:].iterrows():
    p = df['Price'].iloc[i]
    s = df['Share'].iloc[i]
    δ = df['δ'].iloc[i]
    δ_lower = df['δ'].iloc[i-1]
    f = 1 / b
    
    if i < N -2:
        δ_higher = df['δ'].iloc[i+1]
    else:
        δ_higher = float('inf')
    
    df['partials'].iloc[i] = p - s / (f / (δ_higher - δ) + f / (δ - δ_lower) )

<ipython-input-13-4b9e37bf8732>:17: RuntimeWarning: divide by zero encountered in double_scalars
  df['mc NE'].iloc[i] = p - s / (f / (δ_higher - δ) + f / (δ - δ_lower) )


In [14]:
x = df[['Air Conditioning', 'Weight of Car', 'Horsepower', 'Quantity']]
y = df['mc NE']
supply2 = sm.OLS(y, x).fit()

In [15]:
pystout(models=[supply1, supply2],
        file='supply.tex',
        digits=3,
        endog_names=False,
        exogvars=['Air Conditioning', 'Weight of Car', 'Horsepower', 'Quantity'],
        stars=False,
        modstat={'nobs':'Obs','rsquared_adj':'Adj. R\sym{2}'}
        )